In [ ]:
!pip install -U rebound

## **CONTOH MASALAH TIGA BENDA DENGAN CLOSE ENCOUNTER**

REBOUND menghitung jarak minimum antar semua benda tiap step waktu. Jika jarak tersebut kurang dari `sim.exit_min_distance`, maka integrasi dihentikan (dengan memanfaatkan fitur blok eksepsi pada `python`). Akan digunakan kriteria radius Hill sebagai kriteria *close encounter*, yaitu $r_{\rm Hill} \approx a \sqrt{\frac{m}{3M}}$, yaitu kira-kira sebesar 0.15 au pada contoh ini.

In [ ]:
import rebound
import numpy as np

In [ ]:
def setupSimulation():
    sim = rebound.Simulation()

    sim.integrator = "ias15" # IAS15 adalah integrator baku (default), sehingga baris ini tidak ditulis juga tidak masalah

    # yr2pi = year divided by 2pi, with year defined as orbital period of planet at 1AU around 1Msun star --> G = 1
    sim.units = ('au', 'msun', 'yr2pi')

    sim.add(m = 1)
    sim.add(m = 1e-3, a = 1)
    sim.add(m = 5e-3, a = 1.25)

    sim.N_active = sim.N

    sim.move_to_com()
    return sim

In [ ]:
sim = setupSimulation()

# Integrasi 100 kali periode orbit
sim.integrate(100 * 2 * np.pi)

In [ ]:
# explore the final particle orbits

for o in sim.calculate_orbits():
    print(o)

# The **orbits of both planets changed significantly**. Was there **close encounter(s)?**

Redo the simulation.

REBOUND calculates the minimum distance between all particle pairs each timestep. If the distance is less than `sim.exit_min_distance`, then the integration is stopped and an exception thrown. Here, we'll use the Hill radius as the criteria for a close encounter. It is given by $r_{\rm Hill} \approx a \sqrt{\frac{m}{3M}}$, which is approximately 0.15 AU in our case.

In [ ]:
# Resets everything
sim = setupSimulation()

sim.exit_min_distance = 0.15

In [ ]:
Noutputs  = 1000
times     = np.linspace(0, 100 * 2 * np.pi, Noutputs)
distances = np.zeros(Noutputs)

In [ ]:
# ps is now an array of pointers. It will update as the simulation runs.

ps = sim.particles

In [ ]:
try:
    for i, time in enumerate(times):
        sim.integrate(time)

        # Calculates the coponentwise difference between particles
        dp = ps[1] - ps[2]

        distances[i] = np.sqrt(dp.x * dp.x + dp.y * dp.y + dp.z * dp.z)
except rebound.Encounter as error:
    print(error)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
fig = plt.figure(figsize = (10, 5))
ax  = plt.subplot(111)

ax.set_xlabel('time [orbits]')
ax.set_xlim([0, sim.t/(2 * np.pi)])
ax.set_ylabel('distance')

plt.plot(times/(2 * np.pi), distances);
plt.plot([0.0, 12], [0.2, 0.2]); # Plot our close encounter criteria;

plt.savefig('/content/gdrive/My Drive/Colab Notebooks/rebound/encounter.pdf')

In [ ]:
print(f'Current number of particles        = {sim.N:d}')
print(f'Current number of active particles = {sim.N_active:d}')
print(f'Current number of real particles   = {sim.N_real:d}')
print(f'Integrator                         : {sim.integrator}')
print(f'Simulation units                   : {sim.units}')
print(f'Gravity                            : {sim.gravity}')
print(f'Softening                          = {sim.softening}')
print(f'G constant                         = {sim.G:E}')
print(f'Test particle type                 = {sim.testparticle_type}')
print(f'Time                               = {sim.t:E}')
print(f'Timestep                           = {sim.dt:E}')
print(f'Last timestep done                 = {sim.dt_last_done:E}')
print(f'Python walltime                    = {sim.walltime:E}')
print(f'Simulation status                  : {sim.status}')
print(f'Simulation message                 : {sim.messages}')

In [ ]:
sim = None

#### Akibat *close encounter*, terjadi *merger* sehingga jumlah partikel (objek) berkurang. Pada blok di atas, jumlah partikel belum berubah karena perhitungan *merger* belum dimasukkan. Untuk selengkapnya, silakan telaah lebih lanjut dengan mengikuti contoh notebook di dokumentasi online REBOUND [di sini](https://rebound.readthedocs.io/en/latest/ipython_examples/CloseEncounters/).